<a href="https://colab.research.google.com/github/SreeDeekshithaEswaran/FML1/blob/main/Paraphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paraphrasing

In [24]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
!pip install wordnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!pip install synsets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement synsets (from versions: none)
ERROR: No matching distribution found for synsets


In [30]:
!pip install nltk

import nltk
nltk.download('wordnet')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [31]:
import nltk
from nltk.tokenize import word_tokenize
#from nltk.corpus
import wordnet

def paraphrase_sentence(sentence):
    tokens = word_tokenize(sentence)
    paraphrased_tokens = []

    for token in tokens:
        paraphrased_token = get_paraphrased_word(token)
        paraphrased_tokens.append(paraphrased_token)

    paraphrased_sentence = ' '.join(paraphrased_tokens)
    return paraphrased_sentence

def get_paraphrased_word(word):
    synonyms = wordnet.synsets(word)

    if not synonyms:
        return word

    lemma = synonyms[0].lemmas()[0]
    paraphrased_word = lemma.name().replace("_", " ")

    return paraphrased_word

# Example usage
sentence = "I want to go for a walk."
paraphrased_sentence = paraphrase_sentence(sentence)
print(paraphrased_sentence)


AttributeError: ignored

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.8 MB/s eta 0:00:00


In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# ... Rest of the code ...

#def paraphrase(model, sentence, tokenizer, max_length, device):
 #   model.eval()
#  with torch.no_grad():
 #       source_tokens = tokenizer.encode(sentence, return_tensors="pt").to(device)
  #      output_tokens = []
   #     hidden = None
    #    for i in range(max_length):
     #       output, hidden = model.lstm(model.embedding(source_tokens), hidden)
      #      logits = model.linear(output)
       #     softmax_probs = nn.functional.softmax(logits, dim=-1)
        #    next_token = torch.argmax(softmax_probs.squeeze(), dim=-1).item()
         #   if next_token == tokenizer.eos_token_id:
    #            break
     #       output_tokens.append(next_token)
 #           source_tokens = torch.tensor([[next_token]]).to(device)
  #      output_tokens = tokenizer.decode(output_tokens)
   # return output_tokens
#YourTokenizer()

class ParaphraseDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, tokenizer):
        self.source_sentences = source_sentences
        self.target_sentences = target_sentences
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, idx):
        source_sentence = self.source_sentences[idx]
        target_sentence = self.target_sentences[idx]
        source_tokens = self.tokenizer(source_sentence)
        target_tokens = self.tokenizer(target_sentence)
        return source_tokens, target_tokens

class ParaphraseModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded)
        logits = self.linear(output)
        return logits

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        source_tokens, target_tokens = batch
        source_tokens = torch.tensor(source_tokens).to(device)
        target_tokens = torch.tensor(target_tokens).to(device)
        logits = model(source_tokens)
        loss = criterion(logits.view(-1, logits.shape[-1]), target_tokens.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def paraphrase(model, sentence, tokenizer, max_length, device):
    model.eval()
    with torch.no_grad():
        source_tokens = tokenizer(sentence)
        source_tokens = torch.tensor([source_tokens]).to(device)
        output_tokens = []
        hidden = None
        for i in range(max_length):
            output, hidden = model.lstm(model.embedding(source_tokens), hidden)
            logits = model.linear(output)
            softmax_probs = nn.functional.softmax(logits, dim=-1)
            next_token = torch.argmax(softmax_probs.squeeze(), dim=-1).item()
            if next_token == tokenizer.eos_token_id:
                break
            output_tokens.append(next_token)
            source_tokens = torch.tensor([[next_token]]).to(device)
        output_tokens = tokenizer.decode(output_tokens)
    return output_tokens

# Example usage
source_sentences = ["The quick brown fox jumps over the lazy dog", "The cat in the hat is wearing a top hat"]
target_sentences = ["The fast brown fox jumps over the lazy dog", "The cat in the hat is wearing a tall hat"]
#tokenizer = YourTokenizer() # Replace this with your own tokenizer
dataset = ParaphraseDataset(source_sentences, target_sentences, tokenizer)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)
model = ParaphraseModel(len(tokenizer), 100, 200)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(10):
    train_loss = train(model, train_loader,optimizer, criterion, device)
print(f"Epoch {epoch + 1} Loss: {train_loss}")

sentence = "The brown fox jumps over the lazy dog"
paraphrased_sentence = paraphrase(model, sentence, tokenizer, max_length=20, device=device)
print(f"Original sentence: {sentence}")
print(f"Paraphrased sentence: {paraphrased_sentence}")


RuntimeError: ignored

In [42]:
import nltk
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def paraphrase_sentence(sentence):
    paraphrased_sentence = []
    words = nltk.word_tokenize(sentence)
    for word in words:
        synonyms = get_synonyms(word)
        if synonyms:
            paraphrased_sentence.append(synonyms[0])  # Replace with first synonym
        else:
            paraphrased_sentence.append(word)
    return " ".join(paraphrased_sentence)

# Example usage
#sentence = "msft.txt"
file_path = "/content/msft.txt"  # Replace with the actual path of your text file
with open(file_path, 'r') as file:
    sentence = file.read()
paraphrased_sentence = paraphrase_sentence(sentence)
print(f"Original sentence: {sentence}")
print(f"Paraphrased sentence: {paraphrased_sentence}")

Original sentence: In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming l

In [44]:
import nltk
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def paraphrase_sentence(sentence):
    paraphrased_sentence = []
    words = nltk.word_tokenize(sentence)
    for word in words:
        synonyms = get_synonyms(word)
        if synonyms:
            paraphrased_sentence.append(synonyms[0])  # Replace with first synonym
        else:
            paraphrased_sentence.append(word)
    return " ".join(paraphrased_sentence)

# Example usage
#sentence = "msft.txt"
file_path = "/content/Abstract.txt"  # Replace with the actual path of your text file
with open(file_path, 'r') as file:
    sentence = file.read()
paraphrased_sentence = paraphrase_sentence(sentence)
print(f"Original sentence: {sentence}")
print(f"Paraphrased sentence: {paraphrased_sentence}")

Original sentence: Abstract

The Dark web is a part of the internet that isn't visible to the search engines and requires the use of anonymizing browser called Tor to be accessed, along with VPN for increased anonymity .
Though Dark web has a legitimate side, it  is very popular for being a hotbed for criminal activities.
Unlike surface webs,the  Deep and Dark webs has no concept of indexing.
Using these criteria, we came up with an idea of developing a system software, to keep track of the screen,maintain log files with checkpoints and track IP address of the user before the VPN has enabled, inorder to report the user incase of performing any illegal activity.
There is a ML model implemented using NLP and ML Techniques like SVM with RBF kernel, random forests and so on. The website url is feeded to decoder and then to ML model.
If the given output refers to illegal activity, then the user and site is reported to the CyberCrime Intelligence and that specific user will be blocked from a